In [81]:
import folium
import branca.colormap as cm
import networkx as nx
import geojson as gjs
import shapely as sh

from clust_tools import clustering as cl
from clust_tools import spatial as sp

In [2]:
metro = nx.read_gml("data/graphs/metro.gml")
bus = nx.Graph(nx.read_gml("data/graphs/bus.gml").to_undirected())
tram = nx.Graph(nx.read_gml("data/graphs/tram.gml").to_undirected())

metro_nodes = list(metro.nodes(data = True))
bus_nodes = list(bus.nodes(data = True))
tram_nodes = list(tram.nodes(data = True))

In [3]:
print(nx.density(metro))
print(nx.number_of_nodes(metro))
print(nx.number_of_edges(metro))

0.01958670260557053
106
109


In [4]:
print(nx.density(bus))
print(nx.number_of_nodes(bus))
print(nx.number_of_edges(bus))

0.0009721370160382383
2110
2163


In [5]:
print(nx.density(tram))
print(nx.number_of_nodes(tram))
print(nx.number_of_edges(tram))

0.004424514794782055
449
445


In [6]:
metro_regions, _ = cl.cluster_stops(metro_nodes, 2)

In [7]:
metro_points = [x["centroid"] for _, x in metro_regions.items()]
metro_region_neighbors = cl.get_neighbor_list(metro_points, radius=5)

In [8]:
metro_neighbor_graph = nx.Graph(metro_region_neighbors)

In [9]:
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(metro_points)[0], min(metro_points)[1]], [max(metro_points)[0], max(metro_points)[1]]])

for n in metro_neighbor_graph:
    metro_neighbor_graph.nodes[n]["centroid"] = metro_regions[n]["centroid"]
    metro_neighbor_graph.nodes[n]["members"] = metro_regions[n]["members"]
    
    folium.Circle(
        radius = 150,
        location = metro_neighbor_graph.nodes[n]["centroid"],
        popup = "<b>" + str(n) + "</b>" + "<br>" + ", ".join(metro_neighbor_graph.nodes[n]["members"]),
        color = 'blue',
        fill = True,
        fill_color='blue',
        fill_opacity = 0.9
    ).add_to(m)
    
for e in metro_neighbor_graph.to_directed().edges:
    metro_edge_coords = [metro_neighbor_graph.nodes[e[0]]["centroid"], metro_neighbor_graph.nodes[e[1]]["centroid"]]
    folium.PolyLine(metro_edge_coords, color = "blue", weight = 1.3).add_to(m)

In [10]:
linear = cm.linear.Set1_08.scale(0, len(metro_regions)).to_step(len(metro_regions))
linear.caption = "Regions"
m.add_child(linear)

for key, val in metro_regions.items():
    for c, elem in enumerate(val["members"]):

        folium.Circle(
            radius = 100,
            location = val["coords"][c],
            popup = "Region: " + str(key) + "<br>" + elem,
            color = linear(key),
            fill = True,
            fill_color = linear(key),
            fill_opacity=0.3,
        ).add_to(m)
m

In [11]:
# Bus and tram share some stops, some processing is needed before uniting the nets. In the composition, bus attributes take the precedence
bus_tram = nx.compose(tram, bus)

for node in tram.nodes(data = True):
    if node[1]["routes"] != bus_tram.node[node[0]]["routes"]:
        bus_tram.node[node[0]]["routes"] += "," + (node[1]["routes"])

net = nx.union(metro, bus_tram)
stops = list(net.nodes(data = True))
regions, stops_mapping = cl.cluster_stops(stops, 2)

points = [x["centroid"] for _, x in regions.items()]
region_neighbors = cl.get_neighbor_list(points, radius=5)

neighbor_graph = nx.Graph(region_neighbors)

In [12]:
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

for n in neighbor_graph:
    neighbor_graph.nodes[n]["centroid"] = regions[n]["centroid"]
    neighbor_graph.nodes[n]["members"] = regions[n]["members"]
    
    folium.Circle(
        radius = 150,
        location = neighbor_graph.nodes[n]["centroid"],
        popup = "<b>" + str(n) + "</b>" + "<br>" + ", ".join(neighbor_graph.nodes[n]["members"]),
        color = 'blue',
        fill = True,
        fill_color='blue',
        fill_opacity = 0.9
    ).add_to(m)
    
for e in neighbor_graph.to_directed().edges:
    edge_coords = [neighbor_graph.nodes[e[0]]["centroid"], neighbor_graph.nodes[e[1]]["centroid"]]
    folium.PolyLine(edge_coords, color = "blue", weight = 1.3).add_to(m)
    
m

In [79]:
i_points = [p[::-1] for p in points]

# Binding Voronoi's infinite cells
# Method 2: convex hull
hull = scisp.ConvexHull(i_points)
hull_poly = [hull.points[vertex] for vertex in hull.vertices]

boundaries = sh.geometry.Polygon(hull_poly)

vor = sp.voronoi_finite(i_points)
vor_cells = [vor["vertices"][region].tolist() for region in vor["regions"]]

features = []
for elem in vor_cells:
    poly = sh.geometry.Polygon(elem + [elem[0]])
    if poly.overlaps(boundaries):
        poly = poly.intersection(boundaries) 
    features.append(gjs.Feature(geometry = poly))

feature_coll = gjs.FeatureCollection(features)
    
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

linear = cm.linear.Set1_08.scale(0, len(vor["regions"])).to_step(len(vor["regions"]))
m.add_child(linear)

folium.GeoJson(feature_coll).add_to(m)
folium.GeoJson(boundaries).add_to(m)

In [82]:
i_points = [p[::-1] for p in points]

# Binding Voronoi's infinite cells
# Method 3: Upscaled convex hull + poly intersection
hull = scisp.ConvexHull(i_points)
hull_poly = [hull.points[vertex] for vertex in hull.vertices]
boundaries = sh.geometry.Polygon(hull_poly)
boundaries = sh.affinity.scale(boundaries, xfact=1.1, yfact=1.1)

vor = sp.voronoi_finite(i_points)
vor_cells = [vor["vertices"][region].tolist() for region in vor["regions"]]

mp = []
for elem in vor_cells:
    poly = sh.geometry.Polygon(elem)
    if poly.overlaps(boundaries):
        poly = poly.intersection(boundaries) 
    mp.append(poly)

feature_coll = []
for i, pol in enumerate(mp):
    feature_coll.append(gjs.Feature(geometry = pol, id=i))
feature_coll = gjs.FeatureCollection(feature_coll)

m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

linear = cm.linear.Set3_09.scale(0, len(mp)).to_step(len(mp))

folium.GeoJson(feature_coll,
              style_function=lambda feature: {
                'fillColor': linear(feature['id']),
                'color': 'white',
                'weight': 2,
                'dashArray': '10,10',
                'fillOpacity': 0.6,
                }).add_to(m)

for n in neighbor_graph:
    neighbor_graph.nodes[n]["centroid"] = regions[n]["centroid"]
    neighbor_graph.nodes[n]["members"] = regions[n]["members"]
    
    folium.Circle(
        radius = 150,
        location = neighbor_graph.nodes[n]["centroid"],
        popup = "<b>Region: " + str(n) + "</b>",
        color = 'red',
        fill = True,
        fill_color='red',
        fill_opacity = 0.9
    ).add_to(m)

m

In [17]:
c2c_graph = cl.c2c_flow_graph(net, stops_mapping)

In [52]:
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

max_weight = max(dict(c2c_graph.edges).items(), key=lambda x: x[1]['weight'])[1]["weight"]

linear = cm.linear.RdYlGn_11
linear.caption = "Weight"
m.add_child(linear)

for e in c2c_graph.to_directed().edges(data = True):
    weight = c2c_graph[e[0]][e[1]]["weight"]
    edge_coords = [regions[e[0]]["centroid"], regions[e[1]]["centroid"]]
    folium.PolyLine(
        edge_coords,
        color = linear(weight/max_weight),
        weight = 3 * weight/max_weight * 5,
        tooltip = "<b>Weight: " + str(weight) + "</b>" + "<br>"
       ).add_to(m)

for n in c2c_graph:
    
    folium.Circle(
        radius = 150,
        location = regions[n]["centroid"],
        tooltip = "<b>Region: " + str(n) + "</b>" + "<br>",
        color = 'blue',
        fill = True,
        fill_color='blue',
        fill_opacity = 0.9
    ).add_to(m)

m